In [ ]:
# Move to the SPVD directory(SPVD/notebooks)
%cd ..

# Training, Inference, and Testing of the SPVD model.

In [ ]:
# Imports
import torch
import lightning as L
from models import *
from lightning.pytorch.callbacks import ModelCheckpoint
from datasets.shapenet_pointflow_sparse import get_dataloaders
path = "/home/vvrbeast/Desktop/Giannis/Data/ShapeNetCore.v2.PC15k"
path = '/home/tourloid/Desktop/PhD/Data/ShapeNetCore.v2.PC15k/'

#optimization for speed 
torch.set_float32_matmul_precision('medium')

# Parameters

In [ ]:
# This version will not be exported as it will be replaced with argparse
version = 'S' # S, M, L
categories = ['car']
ckpt_name = 'SPVD'
epochs = 2
lr = 0.0001

## Initializing the model

In [ ]:
if version == 'S':
    m = SPVD_S()
elif version == 'M':
    m = SPVD()
elif version == 'L':
    m = SPVD_L()

model = DiffusionBase(m, lr=lr)

## Dataset and Dataloader

In [ ]:
tr_dl, te_dl = get_dataloaders(path, categories)

## Training

In [ ]:
checkpoint_callback = ModelCheckpoint(dirpath='checkpoints/', filename=ckpt_name)

trainer = L.Trainer(
    max_epochs=epochs, 
    gradient_clip_val=10.0, 
    callbacks=[checkpoint_callback]
)

In [ ]:
trainer.fit(model=model, train_dataloaders=tr_dl, val_dataloaders=te_dl)

# Inference

In [ ]:
# Imports
from utils.schedulers import DDPMSparseSchedulerGPU
from utils.visualization import quick_vis_batch
from functools import partial
vis_batch = partial(quick_vis_batch, x_offset = 8, y_offset=8)

In [ ]:
# Sending model to GPU and setting to eval mode:
model = model.cuda().eval()

In [ ]:
ddpm_sched = DDPMSparseSchedulerGPU(n_steps=1000, beta_min=0.0001, beta_max=0.02, pres=1e-5)
preds = ddpm_sched.sample(model.cuda(), 32, 2048)
vis_batch(preds)

# Test

In [ ]:
from test_generation import evaluate_gen

In [ ]:
ddpm_sched = DDPMSparseSchedulerGPU(n_steps=1000, beta_min=0.0001, beta_max=0.02, sigma='coef_bt')
evaluate_gen(path, model, ddpm_sched, save_path='./results/', cates=categories)

In [ ]:
ddpm_sched = DDPMSparseSchedulerGPU(n_steps=1000, beta_min=0.0001, beta_max=0.02, sigma='coef_bt')
evaluate_gen(path, model, ddpm_sched, save_path='./results/', cates=categories)

In [ ]:
ddpm_sched = DDPMSparseSchedulerGPU(n_steps=1000, beta_min=0.0001, beta_max=0.02, sigma='coef_bt')
evaluate_gen(path, model, ddpm_sched, save_path='./results/', cates=categories)